# Evaluation of Mulitple Jets

### Imports

In [ ]:
import os
import sys

sys.path.append("../")

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import numpy as np
import pytorch_lightning as pl
import torch
from omegaconf import OmegaConf

In [ ]:
# set env variable DATA_DIR again because of hydra
from dotenv import load_dotenv

load_dotenv()
os.environ["DATA_DIR"] = os.environ.get("DATA_DIR")

In [ ]:
# plots and metrics
import matplotlib.pyplot as plt

from src.data.components import calculate_all_wasserstein_metrics
from src.utils.data_generation import generate_data
from src.utils.plotting import apply_mpl_styles, create_and_plot_data, plot_single_jets

apply_mpl_styles()

### Load specified model and datamodule from selected experiment

In [ ]:
experiment = "fm_all_jets.yaml"

In [ ]:
# load everything from experiment config
with hydra.initialize(version_base=None, config_path="../configs/"):
    cfg = hydra.compose(config_name="train.yaml", overrides=[f"experiment={experiment}"])
    print(OmegaConf.to_yaml(cfg))

In [ ]:
datamodule = hydra.utils.instantiate(cfg.data)
model = hydra.utils.instantiate(cfg.model)

In [ ]:
model_name_for_saving = "nb_fm_tops30"

In [ ]:
datamodule.setup()

In [ ]:
test_data = np.array(datamodule.tensor_test)
test_mask = np.array(datamodule.mask_test)
test_cond = np.array(datamodule.tensor_conditioning_test)
val_data = np.array(datamodule.tensor_val)
val_mask = np.array(datamodule.mask_val)
val_cond = np.array(datamodule.tensor_conditioning_val)
train_data = np.array(datamodule.tensor_train)
train_mask = np.array(datamodule.mask_train)
train_cond = np.array(datamodule.tensor_conditioning_train)
means = np.array(datamodule.means)
stds = np.array(datamodule.stds)

In [ ]:
print(test_data.shape)
print(test_mask.shape)
print(test_cond.shape)
print(val_data.shape)
print(val_mask.shape)
print(val_cond.shape)
print(train_data.shape)
print(train_mask.shape)
print(train_cond.shape)
print(means)
print(stds)

### Load Checkpoint

In [ ]:
# cond
ckpt = "/beegfs/desy/user/ewencedr/deep-learning/logs/150 alljets nocond onlymetrics/runs/2023-06-14_18-51-51/checkpoints/epoch_1979_w1m_0.00055217-EMA.ckpt"
ckpt = "/beegfs/desy/user/ewencedr/deep-learning/logs/30 alljets nocond onlymetrics/runs/2023-06-14_18-51-38/checkpoints/epoch_2983_w1m_0.00026751-EMA.ckpt"
model = model.load_from_checkpoint(ckpt)

## Generate Combined

In [ ]:
# chosse between test and val
mask_gen = test_mask
data_gen = test_data
cond_gen = test_cond

In [ ]:
# increase size for better statistics
big_mask = np.repeat(mask_gen, 5, axis=0)
big_data = np.repeat(data_gen, 5, axis=0)
big_cond = np.repeat(cond_gen, 5, axis=0)

In [ ]:
data, generation_time = generate_data(
    model,
    num_jet_samples=5 * len(mask_gen),
    batch_size=1000,
    cond=torch.tensor(big_cond),
    variable_set_sizes=True,
    mask=torch.tensor(big_mask),
    normalized_data=True,
    means=means,
    stds=stds,
    ode_solver="midpoint",
    ode_steps=100,
)

In [ ]:
particle_data = data
w_dists_big = calculate_all_wasserstein_metrics(
    data_gen[..., :3],
    particle_data,
    None,
    None,
    num_eval_samples=len(data_gen),
    num_batches=5,
    calculate_efps=True,
    use_masks=False,
)

print(w_dists_big)

## Generate one jet type

In [ ]:
jet_type = "t"
dataset = "test"
if not datamodule.hparams.conditioning_type:
    raise ValueError("Data module has no conditioning on jet type")

In [ ]:
if dataset == "test":
    mask = test_mask
    data = test_data
    cond = test_cond
elif dataset == "val":
    mask = val_mask
    data = val_data
    cond = val_cond
print(mask.shape)
print(data.shape)
print(cond.shape)

In [ ]:
print(train_cond.shape)

In [ ]:
# select only data, mask and cond for the specified jet type
# also for training data because it is compared to test data later
index_jettype = np.squeeze(np.argwhere(np.array(datamodule.jet_types) == jet_type))

indice_jettype = np.squeeze(np.argwhere(cond[:, index_jettype] == 1))
indice_jettype_train = np.squeeze(np.argwhere(train_cond[:, index_jettype] == 1))

mask_jettype = mask[indice_jettype]
data_jettype = data[indice_jettype]
cond_jettype = cond[indice_jettype]
train_mask_jettype = train_mask[indice_jettype_train]
train_data_jettype = train_data[indice_jettype_train]
train_cond_jettype = train_cond[indice_jettype_train]

print(mask_jettype.shape)
print(data_jettype.shape)
print(cond_jettype.shape)
print(train_mask_jettype.shape)
print(train_data_jettype.shape)
print(train_cond_jettype.shape)

In [ ]:
factor = 5

In [ ]:
# increase size for better statistics
big_mask_jettype = np.repeat(mask_jettype, factor, axis=0)
big_data_jettype = np.repeat(data_jettype, factor, axis=0)
big_cond_jettype = np.repeat(cond_jettype, factor, axis=0)

In [ ]:
data, generation_time = generate_data(
    model,
    num_jet_samples=len(big_mask_jettype),
    batch_size=1000,
    cond=torch.tensor(big_cond_jettype),
    variable_set_sizes=True,
    mask=torch.tensor(big_mask_jettype),
    normalized_data=True,
    means=means,
    stds=stds,
    ode_solver="midpoint",
    ode_steps=100,
)

In [ ]:
w_dists_big_jettype = calculate_all_wasserstein_metrics(
    data_jettype[..., :3],
    data,
    None,
    None,
    num_eval_samples=len(data_jettype),
    num_batches=factor,
    calculate_efps=True,
    use_masks=False,
)

print(w_dists_big_jettype)

## Jet substructure

#### Generated data
Using statistics of one val/test dataset

In [ ]:
import h5py
from scipy.stats import wasserstein_distance

from src.data.components.metrics import wasserstein_distance_batched
from src.utils.jet_substructure import dump_hlvs

In [ ]:
path = "/".join(ckpt.split("/")[:-2]) + "/"
file_name = model_name_for_saving + "_" + jet_type + "_substructure"
full_path = path + file_name
print(full_path)

In [ ]:
dump_hlvs(data, full_path, plot=True)
# dump_hlvs(data[: len(data) // factor], full_path, plot=True)

#### JetNet Data

In [ ]:
path_jetnet = f"/beegfs/desy/user/ewencedr/deep-learning/jetnet_substructure/{jet_type}-{datamodule.hparams.num_particles}_substructure"

In [ ]:
dump_hlvs(np.repeat(data_jettype, 5, axis=0), path_jetnet, plot=True)
# dump_hlvs(data_jettype, path_jetnet, plot=True)

#### load and compare generated data with jetnet data (W-Dists)

In [ ]:
keys = []
data_substructure = []
with h5py.File(full_path + ".h5", "r") as f:
    print(f.keys())
    tau21 = np.array(f["tau21"])
    tau32 = np.array(f["tau32"])
    d2 = np.array(f["d2"])
    for key in f.keys():
        keys.append(key)
        data_substructure.append(np.array(f[key]))
data_substructure = np.array(data_substructure)

In [ ]:
data_substructure_jetnet = []
with h5py.File(path_jetnet + ".h5", "r") as f:
    tau21_jetnet = np.array(f["tau21"])
    tau32_jetnet = np.array(f["tau32"])
    d2_jetnet = np.array(f["d2"])
    for key in f.keys():
        data_substructure_jetnet.append(np.array(f[key]))
data_substructure_jetnet = np.array(data_substructure_jetnet)

In [ ]:
w_dist_tau21_mean, w_dist_tau21_std = wasserstein_distance_batched(
    tau21_jetnet[: len(tau21_jetnet) // factor], tau21, num_batches=factor
)
w_dist_tau32_mean, w_dist_tau32_std = wasserstein_distance_batched(
    tau32_jetnet[: len(tau32_jetnet) // factor], tau32, num_batches=factor
)
w_dist_d2_mean, w_dist_d2_std = wasserstein_distance_batched(
    d2_jetnet[: len(d2_jetnet) // factor], d2, num_batches=factor
)
print(f"W-Dist tau21: {w_dist_tau21_mean:4.3E} +- {w_dist_tau21_std:4.3E}")
print(f"W-Dist tau32: {w_dist_tau32_mean:4.3E} +- {w_dist_tau32_std:4.3E}")
print(f"W-Dist d2: {w_dist_d2_mean:4.3E} +- {w_dist_d2_std:4.3E}")

#### truth data

In [ ]:
path_jetnet_train = f"/beegfs/desy/user/ewencedr/deep-learning/jetnet_substructure/{jet_type}-{datamodule.hparams.num_particles}-train_substructure"

In [ ]:
dump_hlvs(train_data_jettype, path_jetnet_train, plot=True)

In [ ]:
with h5py.File(path_jetnet_train + ".h5", "r") as f:
    tau21_jetnet_train = np.array(f["tau21"])
    tau32_jetnet_train = np.array(f["tau32"])
    d2_jetnet_train = np.array(f["d2"])

In [ ]:
print((tau21_jetnet[: len(tau21_jetnet) // factor]).shape)
print(tau21_jetnet_train.shape)

In [ ]:
w_dist_tau21_mean_truth, w_dist_tau21_std_truth = wasserstein_distance_batched(
    tau21_jetnet[: len(tau21_jetnet) // factor], tau21_jetnet_train, num_batches=factor - 1
)
w_dist_tau32_mean_truth, w_dist_tau32_std_truth = wasserstein_distance_batched(
    tau32_jetnet[: len(tau32_jetnet) // factor], tau32_jetnet_train, num_batches=factor - 1
)
w_dist_d2_mean_truth, w_dist_d2_std_truth = wasserstein_distance_batched(
    d2_jetnet[: len(d2_jetnet) // factor], d2_jetnet_train, num_batches=factor - 1
)
print(f"W-Dist tau21: {w_dist_tau21_mean_truth:4.3E} +- {w_dist_tau21_std_truth:4.3E}")
print(f"W-Dist tau32: {w_dist_tau32_mean_truth:4.3E} +- {w_dist_tau32_std_truth:4.3E}")
print(f"W-Dist d2: {w_dist_d2_mean_truth:4.3E} +- {w_dist_d2_std_truth:4.3E}")

#### Histogram Plots for Substructure Metrics

In [ ]:
bins = 100
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

hist_tau21_jetnet = ax1.hist(
    tau21_jetnet, bins=bins, label="JetNet", histtype="stepfilled", alpha=0.5
)
hist_tau21 = ax1.hist(tau21, bins=hist_tau21_jetnet[1], label="EPiC-FM", histtype="step")
ax1.set_title(r"$\tau_{21}$")
ax1.legend(loc="best")

hist_tau32_jetnet = ax2.hist(
    tau32_jetnet, bins=bins, label="JetNet", histtype="stepfilled", alpha=0.5
)
hist_tau32 = ax2.hist(tau32, bins=hist_tau32_jetnet[1], label="EPiC-FM", histtype="step")
ax2.set_title(r"$\tau_{32}$")
ax2.legend(loc="best")

hist_d2_jetnet = ax3.hist(d2_jetnet, bins=bins, label="JetNet", histtype="stepfilled", alpha=0.5)
hist_d2 = ax3.hist(d2, bins=hist_d2_jetnet[1], label="EPiC-FM", histtype="step")
ax3.set_title(r"$d_2$")
ax3.legend(loc="best")

plt.legend(loc="best")
plt.tight_layout()

In [ ]:
bins = 100
fig, axs = plt.subplots(4, 3, figsize=(15, 20))

for i, ax in enumerate(axs.flatten()):
    hist_jetnet = ax.hist(
        data_substructure_jetnet[i], bins=bins, label="JetNet", histtype="stepfilled", alpha=0.5
    )
    hist = ax.hist(data_substructure[i], bins=hist_jetnet[1], label="EPiC-FM", histtype="step")
    ax.set_title(keys[i])
    ax.legend(loc="best")

plt.legend(loc="best")
plt.tight_layout()

### Histograms

In [ ]:
fig, data, generation_times = create_and_plot_data(
    np.array(data_jettype),
    [model],
    cond=torch.tensor(cond_jettype),
    save_name="fm_tops_nb",
    labels=["FM"],
    mask=mask_jettype,
    num_jet_samples=len(data_jettype),
    batch_size=1000,
    variable_set_sizes=True,
    normalized_data=[True],
    means=means,
    stds=stds,
    save_folder="./logs/nb_plots/",
    plottype="sim_data",
    plot_jet_features=True,
    plot_w_dists=False,
    plot_selected_multiplicities=False,
    selected_multiplicities=[1, 3, 5, 10, 20, 30],
    ode_solver="midpoint",
    ode_steps=100,
    bins=100,
    mass_linear=False,
)